<a href="https://colab.research.google.com/github/JohnMasapantaPozo/Machine-and-Deep-Learning-Applied-to-Geosciences/blob/root/Thesis_Final_Submition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing libraries - NOTE: XGB model has to be run in a GPU, otherwise remove "tree_method=gpu_hist" from parameters.
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import max_error, mean_squared_error
from xgboost import XGBRegressor
import warnings 
warnings.filterwarnings('ignore', category=FutureWarning)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, recall_score, precision_score, f1_score

In [2]:
#Run if running in Google Cloud
from google.colab import drive   #Getting access to Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Data Directory
directory = '/content/drive/MyDrive/Thesis_data/'

In [4]:
#Reading Raw Data
def reading_data():
  lithology_numbers = {30000: 0, 
                       65030: 1, 
                       65000: 2, 
                       80000: 3, 
                       74000: 4, 
                       70000: 5, 
                       70032: 6, 
                       88000: 7, 
                       86000: 8, 
                       99000: 9, 
                       90000: 10, 
                       93000: 11
                       }

  #training dataset
  labeled_data = pd.read_csv(directory + 'train.csv', sep=';')
  labeled_data = labeled_data.rename(columns={'FORCE_2020_LITHOFACIES_LITHOLOGY':'LITHO', 
                                              'FORCE_2020_LITHOFACIES_CONFIDENCE':'LIT_CONF'
                                              })
  labeled_data = labeled_data.drop(['LIT_CONF'], axis=1)
  labeled_data['LITHO'] = labeled_data["LITHO"].map(lithology_numbers)

  #open dataset
  test_data = pd.read_csv(directory + 'test.csv', sep=';')
  test_labels = pd.read_csv(directory + 'test_target.csv', sep=';')
  test_data = pd.merge(test_data, test_labels, on=['WELL', 'DEPTH_MD'])
  test_data = test_data.rename(columns={'FORCE_2020_LITHOFACIES_LITHOLOGY':'LITHO'})
  test_data['LITHO'] = test_data["LITHO"].map(lithology_numbers)

  #hidden dataset
  hidden_data = pd.read_csv(directory + 'hidden_test.csv', sep=';')
  hidden_data = hidden_data.rename(columns={'FORCE_2020_LITHOFACIES_LITHOLOGY':'LITHO'})
  hidden_data = hidden_data.drop(['FORCE_2020_LITHOFACIES_CONFIDENCE'], axis=1)
  hidden_data['LITHO'] = hidden_data['LITHO'].map(lithology_numbers)

  return labeled_data, test_data, hidden_data

def base_well_name(row):
    well_name = row['WELL']
    return well_name.split()[0]

def data_preprocessing(training_data, test_data1, test_data2):

  train_len  = training_data.shape[0]; train_well = training_data.WELL.values; train_depth = training_data.DEPTH_MD.values
  test_len   = test_data1.shape[0];    test_well = test_data1.WELL.values;     test_depth = test_data1.DEPTH_MD.values
  hidden_len = test_data2.shape[0];    hidden_well = test_data2.WELL.values;   hidden_depth = test_data2.DEPTH_MD.values

  #concatenate datasets
  df_concat = pd.concat((training_data, test_data1, test_data2)).reset_index(drop=True)

  #dropping columns
  drop_cols = ['SGR', 'ROPA', 'RXO', 'MUDWEIGHT']
  df_drop = df_concat.drop(drop_cols, axis=1)

  #encoding datasets
  df_drop['GROUP_encoded'] = df_drop['GROUP'].astype('category')
  df_drop['GROUP_encoded'] = df_drop['GROUP_encoded'].cat.codes

  df_drop['FORMATION_encoded'] = df_drop['FORMATION'].astype('category')
  df_drop['FORMATION_encoded'] = df_drop['FORMATION_encoded'].cat.codes

  df_drop['WELL_encoded'] = df_drop['WELL'].astype('category')
  df_drop['WELL_encoded'] = df_drop['WELL_encoded'].cat.codes

  #clustering by well location
  training_wells = training_data['WELL'].unique(); test_wells = test_data1['WELL'].unique(); hidden_wells = test_data2['WELL'].unique()

  well_names = np.concatenate((training_wells, test_wells, hidden_wells))
  well_names_df = pd.DataFrame({'WELL':well_names})

  #importing metadata
  well_meta_df = pd.read_csv('/content/drive/MyDrive/Thesis_data/wellbore_exploration_all.csv')
  well_meta_df.rename(columns={'wlbWellboreName': 'WELL', 
                               'wlbWell': 'WELL_HEAD', 
                               'wlbNsDecDeg': 'lat', 
                               'wlbEwDesDeg': 'lon', 
                               'wlbDrillingOperator': 'Drilling_Operator', 
                               'wlbPurposePlanned': 'Purpose', 
                               'wlbCompletionYear': 'Completion_Year', 
                               'wlbFormationAtTd': 'Formation'
                               }, inplace=True)
  
  well_locations_df = well_meta_df[['WELL_HEAD', 'lat', 'lon']].drop_duplicates(subset=['WELL_HEAD'])
  well_meta_df = well_meta_df[['WELL', 'Drilling_Operator', 'Purpose', 'Completion_Year', 'Formation']]

  well_names_df['WELL_HEAD'] = well_names_df.apply(lambda row: base_well_name(row), axis=1)
  locations_df = well_names_df.merge(well_locations_df, how='inner', on='WELL_HEAD')
  locations_df = locations_df.merge(well_meta_df, how='left', on='WELL')

  #Labeling train and test wells
  locations_df.loc[locations_df['WELL'].isin(training_wells), 'Dataset'] = 'Train'
  locations_df.loc[locations_df['WELL'].isin(test_wells), 'Dataset'] = 'Test'
  locations_df.loc[locations_df['WELL'].isin(hidden_wells), 'Dataset'] = 'Hidden'

  LonLat_df =  locations_df.drop(['WELL', 
                                  'WELL_HEAD', 
                                  'Drilling_Operator', 
                                  'Purpose', 
                                  'Completion_Year', 
                                  'Formation', 
                                  'Dataset'], 
                                  axis=1)
  
  location = LonLat_df[['lon', 'lat']].values
  kmeans = KMeans(n_clusters=3, init='k-means++', random_state=1)
  labels = kmeans.fit_predict(location)

  df_drop = df_drop.rename(columns={'WELL':'Cluster'})
  clust_map = dict(zip(locations_df.WELL.values, labels))
  df_drop['Cluster'] = df_drop['Cluster'].map(clust_map)
  df_drop2 = df_drop.drop(['GROUP', 'FORMATION'], axis=1)

  #spliting datasets
  traindata = df_drop2[:train_len].copy()
  testdata = df_drop2[train_len:(train_len+test_len)].copy()
  hiddendata = df_drop2[(train_len+test_len):].copy()

  return traindata, testdata, hiddendata

In [5]:
def data_augmentation(traindata, testdata, hiddendata):

  ##1. PREDICTING DTS

  print('-----------------------------------------PREDINCTING DTS------------------------------------------')

  traindata_dts = traindata[traindata.DTS.notna()]
  X_dts = traindata_dts.drop(['LITHO', 'DTS'], axis=1)
  Y_dts = traindata_dts['DTS']
  X_dts_inp = X_dts.apply(lambda x: x.fillna(x.median()), axis=0)    #Imputation
  X_dts_train, X_dts_val, Y_dts_train, Y_dts_val = train_test_split(X_dts_inp, Y_dts, test_size=0.3, random_state=42)   #train validation split

  print('Training set shape {} and validation set shape {}'.format(X_dts_train.shape, X_dts_val.shape))

  model1000 = XGBRegressor('''
  
                                HIDEN PARAMETERS
                           '''
                          )
  
  model1000.fit(X_dts_train, Y_dts_train.values.ravel(), early_stopping_rounds=100, eval_set=[(X_dts_val, Y_dts_val)], verbose=100)
  train_pred = model1000.predict(X_dts_train)
  val_pred = model1000.predict(X_dts_val)

  print('Train set root mean squared error:', np.sqrt(mean_squared_error(Y_dts_train, train_pred)))
  print('Validation set root mean squared error:', np.sqrt(mean_squared_error(Y_dts_val, val_pred)))

  print('--------------------------------Imputing DTS LOG by ML predictions--------------------------------')
  # Filling nan values before predicting DTS
  X_train_DTS = traindata.drop(['LITHO', 'DTS'], axis=1)
  X_train_DTS2 = X_train_DTS.apply(lambda x: x.fillna(x.median()), axis=0)

  X_test_DTS = testdata.drop(['LITHO', 'DTS'], axis=1)
  X_test_DTS2 = X_test_DTS.apply(lambda x: x.fillna(x.median()), axis=0)

  X_hidden_DTS = hiddendata.drop(['LITHO', 'DTS'], axis=1)
  X_hidden_DTS2 = X_hidden_DTS.apply(lambda x: x.fillna(x.median()), axis=0)

  #Predicting DTS (COMPLETE DATASETS)
  traindata['DTS_pred'] = model1000.predict(X_train_DTS2)
  testdata['DTS_pred'] = model1000.predict(X_test_DTS2)
  hiddendata['DTS_pred'] = model1000.predict(X_hidden_DTS2)

  #Inputing nan values in DTS with DTS_PREDICTED
  traindata['DTS_COMB'] = traindata['DTS']
  traindata['DTS_COMB'].fillna(traindata['DTS_pred'], inplace=True)

  testdata['DTS_COMB'] = testdata['DTS']
  testdata['DTS_COMB'].fillna(testdata['DTS_pred'], inplace=True)

  hiddendata['DTS_COMB'] = hiddendata['DTS']
  hiddendata['DTS_COMB'].fillna(hiddendata['DTS_pred'], inplace=True)
  
  ##2. PREDICTING NPHI

  print('----------------------------------------PREDINCTING NPHI-----------------------------------------')

  traindata_nphi = traindata[traindata.NPHI.notna()]
  X_nphi = traindata_nphi.drop(['LITHO', 'DTS', 'DTS_pred', 'NPHI'], axis=1)
  Y_nphi = traindata_nphi['NPHI']
  X_nphi_inp = X_nphi.apply(lambda x: x.fillna(x.median()), axis=0)     #Imputation
  X_nphi_train, X_nphi_val, Y_nphi_train, Y_nphi_val = train_test_split(X_nphi_inp, Y_nphi, test_size=0.3, random_state=42)     #train-validation split

  print('Training set sahpe {} and validation set shape {}'.format(X_nphi_train.shape, X_nphi_val.shape))

  model2000 = XGBRegressor('''
  
                                HIDEN PARAMETERS
                           '''
                          )
  
  model2000.fit(X_nphi_train, Y_nphi_train.values.ravel(), early_stopping_rounds=100, eval_set=[(X_nphi_val, Y_nphi_val)], verbose=100)
  train_pred = model2000.predict(X_nphi_train)
  val_pred = model2000.predict(X_nphi_val)

  print('Training set root mean squared error:', np.sqrt(mean_squared_error(Y_nphi_train, train_pred)))
  print('Validation set root mean squared error:', np.sqrt(mean_squared_error(Y_nphi_val, val_pred)))

  print('--------------------------------Imputing NPHI LOG by ML predictions--------------------------------')

  # Filling nan values before predicting NPHI
  X_train_NPHI = traindata.drop(['LITHO', 'DTS', 'DTS_pred', 'NPHI'], axis=1)
  X_train_NPHI2 = X_train_NPHI.apply(lambda x: x.fillna(x.median()), axis=0)

  X_test_NPHI = testdata.drop(['LITHO', 'DTS', 'DTS_pred', 'NPHI'], axis=1)
  X_test_NPHI2 = X_test_NPHI.apply(lambda x: x.fillna(x.median()), axis=0)

  X_hidden_NPHI = hiddendata.drop(['LITHO', 'DTS', 'DTS_pred', 'NPHI'], axis=1)
  X_hidden_NPHI2 = X_hidden_NPHI.apply(lambda x: x.fillna(x.median()), axis=0)

  #Predicting DTS (COMPLETE DATASETS)
  traindata['NPHI_pred'] = model2000.predict(X_train_NPHI2)
  testdata['NPHI_pred'] = model2000.predict(X_test_NPHI2)
  hiddendata['NPHI_pred'] = model2000.predict(X_hidden_NPHI2)

  #Inputing nan values in DTS with DTS_PREDICTED
  traindata['NPHI_COMB'] = traindata['NPHI']
  traindata['NPHI_COMB'].fillna(traindata['NPHI_pred'], inplace=True)

  testdata['NPHI_COMB'] = testdata['NPHI']
  testdata['NPHI_COMB'].fillna(testdata['NPHI_pred'], inplace=True)

  hiddendata['NPHI_COMB'] = hiddendata['NPHI']
  hiddendata['NPHI_COMB'].fillna(hiddendata['NPHI_pred'], inplace=True)

  ##3. PREDICTING RHOB
  print('----------------------------------------PREDINCTING RHOB-----------------------------------------')

  traindata_rhob = traindata[traindata.RHOB.notna()]
  X_rhob = traindata_rhob.drop(['LITHO', 'DTS', 'DTS_pred', 'NPHI', 'NPHI_pred', 'RHOB'], axis=1)
  Y_rhob = traindata_rhob['RHOB']
  X_rhob_inp = X_rhob.apply(lambda x: x.fillna(x.median()), axis=0)    #Imputation
  X_rhob_train, X_rhob_val, Y_rhob_train, Y_rhob_val = train_test_split(X_rhob_inp, Y_rhob, test_size=0.3, random_state=42)  #train-validation split

  print('Training set sahpe {} and validation set shape {}'.format(X_rhob_train.shape, X_rhob_val.shape))

  model4000 = XGBRegressor('''
  
                                HIDEN PARAMETERS
                           '''
                          )
  
  model4000.fit(X_rhob_train, Y_rhob_train.values.ravel(), early_stopping_rounds=100, eval_set=[(X_rhob_val, Y_rhob_val)], verbose=100)
  train_pred = model4000.predict(X_rhob_train)
  val_pred = model4000.predict(X_rhob_val)

  print('Train set root mean squared error:', np.sqrt(mean_squared_error(Y_rhob_train, train_pred)))
  print('Validation set root mean squared error:', np.sqrt(mean_squared_error(Y_rhob_val, val_pred)))
  
  print('--------------------------------Imputing RHOB LOG by ML predictions--------------------------------')

  # Filling nan values before predicting NPHI
  X_train_RHOB = traindata.drop(['LITHO', 'DTS', 'DTS_pred', 'NPHI', 'NPHI_pred', 'RHOB'], axis=1)
  X_train_RHOB2 = X_train_RHOB.apply(lambda x: x.fillna(x.median()), axis=0)

  X_test_RHOB = testdata.drop(['LITHO', 'DTS', 'DTS_pred', 'NPHI', 'NPHI_pred', 'RHOB'], axis=1)
  X_test_RHOB2 = X_test_RHOB.apply(lambda x: x.fillna(x.median()), axis=0)

  X_hidden_RHOB = hiddendata.drop(['LITHO', 'DTS', 'DTS_pred', 'NPHI', 'NPHI_pred', 'RHOB'], axis=1)
  X_hidden_RHOB2 = X_hidden_RHOB.apply(lambda x: x.fillna(x.median()), axis=0)

  #Predicting DTS (COMPLETE DATASETS)
  traindata['RHOB_pred'] = model4000.predict(X_train_RHOB2)
  testdata['RHOB_pred'] = model4000.predict(X_test_RHOB2)
  hiddendata['RHOB_pred'] = model4000.predict(X_hidden_RHOB2)

  #Inputing nan values in DTS with DTS_PREDICTED
  traindata['RHOB_COMB'] = traindata['RHOB']
  traindata['RHOB_COMB'].fillna(traindata['RHOB_pred'], inplace=True)

  testdata['RHOB_COMB'] = testdata['RHOB']
  testdata['RHOB_COMB'].fillna(testdata['RHOB_pred'], inplace=True)

  hiddendata['RHOB_COMB'] = hiddendata['RHOB']
  hiddendata['RHOB_COMB'].fillna(hiddendata['RHOB_pred'], inplace=True)

  ##4. PREDICTING DTC
  print('----------------------------------------PREDINCTING DTC-----------------------------------------')

  traindata_dtc = traindata[traindata.DTC.notna()]
  X_dtc = traindata_dtc.drop(['LITHO', 'DTS', 'DTS_pred', 'NPHI', 'NPHI_pred', 'RHOB', 'RHOB_pred', 'DTC'], axis=1)
  Y_dtc = traindata_dtc['DTC']
  X_dtc_inp = X_dtc.apply(lambda x: x.fillna(x.median()), axis=0)      #Imputation
  X_dtc_train, X_dtc_val, Y_dtc_train, Y_dtc_val = train_test_split(X_dtc_inp, Y_dtc, test_size=0.3, random_state=42)     #Spliting train-validation

  print('Training set sahpe {} and validation set shape {}'.format(X_dtc_train.shape, X_dtc_val.shape))

  model3000 = XGBRegressor('''
  
                                HIDEN PARAMETERS
                           '''
                          )
  
  model3000.fit(X_dtc_train, Y_dtc_train.values.ravel(), early_stopping_rounds=100, eval_set=[(X_dtc_val, Y_dtc_val)], verbose=100)
  train_pred = model3000.predict(X_dtc_train)
  val_pred = model3000.predict(X_dtc_val)

  print('Train set root mean squared error:', np.sqrt(mean_squared_error(Y_dtc_train, train_pred)))
  print('Validation set root mean squared error:', np.sqrt(mean_squared_error(Y_dtc_val, val_pred)))

  print('--------------------------------Imputing DTC LOG by ML predictions--------------------------------')

  # Filling nan values before predicting NPHI
  X_train_DTC = traindata.drop(['LITHO', 'DTS', 'DTS_pred', 'NPHI', 'NPHI_pred', 'RHOB', 'RHOB_pred', 'DTC'], axis=1)
  X_train_DTC2 = X_train_DTC.apply(lambda x: x.fillna(x.median()), axis=0)

  X_test_DTC = testdata.drop(['LITHO', 'DTS', 'DTS_pred', 'NPHI', 'NPHI_pred', 'RHOB', 'RHOB_pred', 'DTC'], axis=1)
  X_test_DTC2 = X_test_DTC.apply(lambda x: x.fillna(x.median()), axis=0)

  X_hidden_DTC = hiddendata.drop(['LITHO', 'DTS', 'DTS_pred', 'NPHI', 'NPHI_pred', 'RHOB', 'RHOB_pred', 'DTC'], axis=1)
  X_hidden_DTC2 = X_hidden_DTC.apply(lambda x: x.fillna(x.median()), axis=0)

  #Predicting DTS (COMPLETE DATASETS)
  traindata['DTC_pred'] = model3000.predict(X_train_DTC2)
  testdata['DTC_pred'] = model3000.predict(X_test_DTC2)
  hiddendata['DTC_pred'] = model3000.predict(X_hidden_DTC2)

  #Inputing nan values in DTS with DTS_PREDICTED
  traindata['DTC_COMB'] = traindata['DTC']
  traindata['DTC_COMB'].fillna(traindata['DTC_pred'], inplace=True)

  testdata['DTC_COMB'] = testdata['DTC']
  testdata['DTC_COMB'].fillna(testdata['DTC_pred'], inplace=True)

  hiddendata['DTC_COMB'] = hiddendata['DTC']
  hiddendata['DTC_COMB'].fillna(hiddendata['DTC_pred'], inplace=True)

  #additional features
  print('--------------------------------Creating additional features--------------------------------')
  #Train Set
  traindata['AI'] = traindata.RHOB * (1e6/traindata.DTS_COMB)
  traindata['AI_P'] = traindata.RHOB * (1e6/traindata.DTC)
  traindata['DT_R'] = traindata.DTC / traindata.DTS_COMB
  traindata['GM'] = ((1e6/traindata.DTS_COMB)**2) * traindata.RHOB
  traindata['K'] = (((1e6/traindata.DTC)**2) * traindata.RHOB) - (4 * traindata.GM/3)
  traindata['MD_TVD'] = -(traindata.DEPTH_MD/traindata.Z_LOC)

  #Test Set
  testdata['AI'] = testdata.RHOB * (1e6/testdata.DTS_COMB)
  testdata['AI_P'] = testdata.RHOB * (1e6/testdata.DTC)
  testdata['DT_R'] = testdata.DTC / testdata.DTS_COMB
  testdata['GM'] = ((1e6/testdata.DTS_COMB)**2) * testdata.RHOB
  testdata['K'] = (((1e6/testdata.DTC)**2) * testdata.RHOB) - (4 * testdata.GM/3)
  testdata['MD_TVD'] = -(testdata.DEPTH_MD/testdata.Z_LOC)

  #Hidden Set
  hiddendata['AI'] = hiddendata.RHOB * (1e6/hiddendata.DTS_COMB)
  hiddendata['AI_P'] = hiddendata.RHOB * (1e6/hiddendata.DTC)
  hiddendata['DT_R'] = hiddendata.DTC / hiddendata.DTS_COMB
  hiddendata['GM'] = ((1e6/hiddendata.DTS_COMB)**2) * hiddendata.RHOB
  hiddendata['K'] = (((1e6/hiddendata.DTC)**2) * hiddendata.RHOB) - (4 * hiddendata.GM/3)
  hiddendata['MD_TVD'] = -(hiddendata.DEPTH_MD/hiddendata.Z_LOC)

  #dropping unnecessary data
  cols_drop = ['DTS_pred', 'NPHI_pred', 'RHOB_pred', 'DTC_pred']
  traindata = traindata.drop(cols_drop, axis=1)
  testdata = testdata.drop(cols_drop, axis=1)
  hiddendata = hiddendata.drop(cols_drop, axis=1)

  print('Features included in the datasets: {}'.format(traindata.columns))

  return traindata, testdata, hiddendata

def normalization(traindata, testdata, hiddendata):

  train_features = traindata.drop(['LITHO'], axis=1);   train_labels = traindata['LITHO']
  test_features = testdata.drop(['LITHO'], axis=1);     test_labels = testdata['LITHO']
  hidden_features = hiddendata.drop(['LITHO'], axis=1); hidden_labels = hiddendata['LITHO']

  train_features_inp = train_features.apply(lambda x: x.fillna(x.median()), axis=0)
  test_features_inp = test_features.apply(lambda x: x.fillna(x.median()), axis=0)
  hidden_features_inp = hidden_features.apply(lambda x: x.fillna(x.median()), axis=0)

  n = train_features_inp.shape[1]
  std = StandardScaler()
  x_train_std = train_features_inp.copy()
  x_test_std = test_features_inp.copy()
  x_hidden_std = hidden_features_inp.copy()

  x_train_std.iloc[:,:n] = std.fit_transform(x_train_std.iloc[:,:n])
  x_test_std.iloc[:,:n] = std.transform(x_test_std.iloc[:,:n])
  x_hidden_std.iloc[:,:n] = std.transform(x_hidden_std.iloc[:,:n])

  cleaned_traindata = pd.concat([x_train_std, train_labels], axis=1)
  cleaned_testdata = pd.concat([x_test_std, test_labels], axis=1)
  cleaned_hiddendata = pd.concat([x_hidden_std, hidden_labels], axis=1)

  return cleaned_traindata, cleaned_testdata, cleaned_hiddendata

# Evaluate prediction
A = np.load(directory + 'penalty_matrix.npy')
def score(y_true, y_pred):
    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]

def predict_litho(clean_traindata, clean_testdata, clean_hiddendata):
  #Recursive Feature Selected Features
  best_features = ['RDEP','GR', 'NPHI_COMB', 'GM', 'AI_P', 'AI', 'DTC', 'DTS_COMB', 'RSHA', 'DT_R', 'RHOB', 
                  'K', 'DCAL', 'Y_LOC', 'Cluster', 'GROUP_encoded', 'WELL_encoded', 'FORMATION_encoded', 
                  'DEPTH_MD', 'Z_LOC', 'CALI', 'BS', 'X_LOC', 'RMED', 'PEF', 'SP', #'MD_TVD', 'ROP','RMIC','DRHO'
                  ]
  #selecting training and test data
  x_train = clean_traindata[best_features]; y_train = clean_traindata['LITHO']
  x_test = clean_testdata[best_features]; y_test = clean_testdata['LITHO']
  x_hidden = clean_hiddendata[best_features]; y_hidden = clean_hiddendata['LITHO']

  #Stratified Cross-validation
  split = 10
  kf = StratifiedKFold(n_splits=split, shuffle=True)

  train_prob_xgb1 = np.zeros((len(x_train), 12))
  open_prob_xgb1 = np.zeros((len(x_test), 12))
  hidden_prob_xgb1 = np.zeros((len(x_hidden), 12))

  xgbmodel_noarg = XGBClassifier('''
  
                                HIDEN PARAMETERS'''

                                )
  i = 1
  for (train_index, test_index) in kf.split(x_train, y_train):
    X_train, X_test = x_train.iloc[train_index], x_train.iloc[test_index]
    Y_train, Y_test = y_train.iloc[train_index], y_train.iloc[test_index]

    xgbmodel_noarg.fit(X_train, Y_train.values.ravel(), early_stopping_rounds=100, eval_set=[(X_test, Y_test)], verbose=100)
    prediction = xgbmodel_noarg.predict(X_test)
    print('Fold accuracy:', accuracy_score(Y_test, prediction))

    print(f'-----------------------FOLD {i}---------------------')
    
    train_prob_xgb1 += xgbmodel_noarg.predict_proba(x_train)
    open_prob_xgb1 += xgbmodel_noarg.predict_proba(x_test)
    hidden_prob_xgb1 += xgbmodel_noarg.predict_proba(x_hidden)

    i += 1

  train_prob_xgb1 = pd.DataFrame(train_prob_xgb1/split)
  train_pred_xgb1 = np.array(pd.DataFrame(train_prob_xgb1).idxmax(axis=1))

  open_prob_xgb1 = pd.DataFrame(open_prob_xgb1/split)
  open_pred_xgb1 = np.array(pd.DataFrame(open_prob_xgb1).idxmax(axis=1))

  hidden_prob_xgb1 = pd.DataFrame(hidden_prob_xgb1/split)
  hidden_pred_xgb1 = np.array(pd.DataFrame(hidden_prob_xgb1).idxmax(axis=1))

  #Printing Reports 
  print('-----------------------TRAIN SET REPORT---------------------')
  print("Open set RMSE:", np.sqrt(mean_squared_error(y_train, train_pred_xgb1)))
  print('Open set penalty matrix score:', score(y_train.values, train_pred_xgb1))
  print('Open set report:', classification_report(y_train, train_pred_xgb1))
  print('-----------------------OPEN SET REPORT---------------------')
  print("Open set RMSE:", np.sqrt(mean_squared_error(y_test, open_pred_xgb1)))
  print('Open set penalty matrix score:', score(y_test.values, open_pred_xgb1))
  print('Open set report:', classification_report(y_test, open_pred_xgb1))
  print('-----------------------HIDDEN SET REPORT---------------------')
  print("Hidden set RMSE:", np.sqrt(mean_squared_error(y_hidden, hidden_pred_xgb1)))
  print('Hidden set penalty matrix score:', score(y_hidden.values, hidden_pred_xgb1))
  print('Hidden set report:', classification_report(y_hidden, hidden_pred_xgb1))

def run_model():
  xxx, yyy, zzz = reading_data()
  traindata, testdata, hiddendata = data_preprocessing(xxx, yyy, zzz)
  a,b,c = data_augmentation(traindata, testdata, hiddendata)
  q, p, t = normalization(a, b, c)
  predict_litho(q, p, t)

In [10]:
## Run the model
run_model()

-----------------------------------------PREDINCTING DTS------------------------------------------
Training set shape (122229, 23) and validation set shape (52384, 23)
[05:53:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:194.561
Will train until validation_0-rmse hasn't improved in 100 rounds.
[99]	validation_0-rmse:15.8819
Train set root mean squared error: 15.833977397169491
Validation set root mean squared error: 15.881878499415139
--------------------------------Imputing DTS LOG by ML predictions--------------------------------
----------------------------------------PREDINCTING NPHI-----------------------------------------
Training set sahpe (535786, 23) and validation set shape (229623, 23)
[05:53:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.194089
Will train until validation_0-rmse hasn'

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Open set report:               precision    recall  f1-score   support

           0       0.78      0.84      0.81     24048
           1       0.56      0.28      0.37     17558
           2       0.83      0.94      0.88     83975
           3       0.71      0.14      0.24      3306
           4       0.00      0.00      0.00       416
           5       0.52      0.52      0.52      4798
           6       0.00      0.00      0.00       625
           8       0.00      0.00      0.00       125
           9       0.77      0.54      0.63      1245
          10       0.83      0.41      0.55       690

    accuracy                           0.79    136786
   macro avg       0.50      0.37      0.40    136786
weighted avg       0.76      0.79      0.76    136786

-----------------------HIDDEN SET REPORT---------------------
Hidden set RMSE: 1.0298652343272097
Hidden set penalty matrix score: -0.43702562154301167
Hidden set report:               precision    recall  f1-score   support